# py_agent: Complete Tutorial - From Basics to Advanced

**py_agent** is an AI-powered forecasting agent that automatically:
- Analyzes your data
- Recommends appropriate models (from 23 options)
- Generates preprocessing pipelines (51 steps)
- Compares multiple models with cross-validation
- Learns from similar forecasting examples
- Autonomously improves workflows until target performance

This tutorial covers all features from **Phases 1, 2, and 3**.

## Setup

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime, timedelta

# Import py_agent
from py_agent import ForecastAgent

# Set random seed for reproducibility
np.random.seed(42)

print("✅ Imports successful")

## Generate Sample Data

We'll create realistic daily sales data with:
- **Trend**: Increasing over time
- **Seasonality**: Weekly pattern (higher on weekends)
- **Features**: Temperature and promotion indicators

In [ ]:
# Generate 2 years of daily data
dates = pd.date_range('2022-01-01', periods=730, freq='D')
n = len(dates)

# Time variable
t = np.arange(n)

# Components
trend = 1000 + 2 * t  # Increasing trend
seasonality = 300 * np.sin(2 * np.pi * t / 7)  # Weekly seasonality
temperature = 20 + 10 * np.sin(2 * np.pi * t / 365) + np.random.randn(n) * 3  # Yearly temp cycle
promotion = np.random.choice([0, 1], n, p=[0.85, 0.15])  # 15% promotion days
promotion_effect = promotion * 500  # Promotions boost sales
noise = np.random.randn(n) * 100  # Random noise

sales = trend + seasonality + 5 * temperature + promotion_effect + noise

# Create DataFrame
data = pd.DataFrame({
    'date': dates,
    'sales': sales,
    'temperature': temperature,
    'promotion': promotion
})

# Split into train/test
split_date = '2023-09-01'
train = data[data['date'] < split_date].copy()
test = data[data['date'] >= split_date].copy()

print(f"Train: {len(train)} days ({train['date'].min()} to {train['date'].max()})")
print(f"Test:  {len(test)} days ({test['date'].min()} to {test['date'].max()})")

# Visualize
plt.figure(figsize=(14, 6))
plt.plot(train['date'], train['sales'], label='Train', alpha=0.7)
plt.plot(test['date'], test['sales'], label='Test', alpha=0.7, color='orange')
plt.axvline(pd.to_datetime(split_date), color='red', linestyle='--', label='Train/Test Split')
plt.xlabel('Date')
plt.ylabel('Sales')
plt.title('Daily Sales Data (Train/Test Split)')
plt.legend()
plt.grid(True, alpha=0.3)
plt.show()

## Phase 1: Basic Workflow Generation

The simplest way to use py_agent: **rule-based workflow generation** (no API costs).

In [ ]:
# Initialize agent (rule-based mode)
agent = ForecastAgent(verbose=True)

# Generate workflow from natural language
workflow = agent.generate_workflow(
    data=train,
    request="Forecast daily sales with weekly seasonality and promotional effects"
)

print("\n" + "="*60)
print("Workflow generated successfully!")
print("="*60)

In [ ]:
# Fit workflow on training data
fit = workflow.fit(train)

# Evaluate on test data
fit_eval = fit.evaluate(test)

# Extract outputs
outputs, coefficients, stats = fit_eval.extract_outputs()

# Display performance
print("\n📊 Performance Metrics:")
test_stats = stats[stats['split'] == 'test']
print(f"  RMSE: {test_stats['rmse'].iloc[0]:.2f}")
print(f"  MAE:  {test_stats['mae'].iloc[0]:.2f}")
print(f"  R²:   {test_stats['r_squared'].iloc[0]:.4f}")

In [ ]:
# Visualize predictions
test_outputs = outputs[outputs['split'] == 'test']

plt.figure(figsize=(14, 6))
plt.plot(test['date'].values, test_outputs['actuals'].values, label='Actual', alpha=0.7)
plt.plot(test['date'].values, test_outputs['fitted'].values, label='Predicted', alpha=0.7)
plt.xlabel('Date')
plt.ylabel('Sales')
plt.title('Phase 1: Basic Workflow - Predictions vs Actuals')
plt.legend()
plt.grid(True, alpha=0.3)
plt.show()

## Phase 3.3: Multi-Model Comparison

Instead of trying one model, **automatically compare 5+ models** with cross-validation.

**Time savings**: 1-2 hours of manual work → 5 minutes automated (96% reduction)

In [ ]:
# Compare top 5 models automatically
results = agent.compare_models(
    data=train,
    request="Forecast daily sales with seasonality",
    n_models=5,
    cv_strategy='time_series',
    n_folds=5,
    date_column='date',
    return_ensemble=True
)

print("\n" + "="*60)
print("Multi-model comparison complete!")
print("="*60)

In [ ]:
# Display rankings
print("\n🏆 Model Rankings (by RMSE):")
print(results['rankings'][['wflow_id', 'mean', 'std_err']].head())

# Best model
print(f"\n✅ Best Model: {results['best_model_id']}")

# Ensemble recommendation
if 'ensemble_recommendation' in results:
    ensemble = results['ensemble_recommendation']
    print(f"\n🤝 Ensemble Recommendation:")
    print(f"  Models: {', '.join(ensemble['model_ids'])}")
    print(f"  Expected RMSE: {ensemble['expected_performance']:.2f}")
    print(f"  Diversity Score: {ensemble['diversity_score']:.2f}")

In [ ]:
# Fit best model on full training data
best_workflow = results['workflowset'][results['best_model_id']]
best_fit = best_workflow.fit(train)
best_eval = best_fit.evaluate(test)

outputs_best, _, stats_best = best_eval.extract_outputs()
test_stats_best = stats_best[stats_best['split'] == 'test']

print(f"\n📊 Best Model Performance on Test Set:")
print(f"  Model: {results['best_model_id']}")
print(f"  RMSE: {test_stats_best['rmse'].iloc[0]:.2f}")
print(f"  MAE:  {test_stats_best['mae'].iloc[0]:.2f}")
print(f"  R²:   {test_stats_best['r_squared'].iloc[0]:.4f}")

In [ ]:
# Visualize best model predictions
test_outputs_best = outputs_best[outputs_best['split'] == 'test']

plt.figure(figsize=(14, 6))
plt.plot(test['date'].values, test_outputs_best['actuals'].values, label='Actual', alpha=0.7)
plt.plot(test['date'].values, test_outputs_best['fitted'].values, label='Best Model Prediction', alpha=0.7)
plt.xlabel('Date')
plt.ylabel('Sales')
plt.title(f'Phase 3.3: Best Model ({results["best_model_id"]}) - Predictions vs Actuals')
plt.legend()
plt.grid(True, alpha=0.3)
plt.show()

## Phase 3.4: RAG Knowledge Base

Get **example-driven recommendations** from similar forecasting scenarios.

The agent searches a knowledge base of 8+ forecasting examples and recommends models that worked in similar cases.

In [ ]:
# Initialize agent with RAG enabled
agent_rag = ForecastAgent(verbose=True, use_rag=True)

# Generate workflow with RAG enhancement
workflow_rag = agent_rag.generate_workflow(
    data=train,
    request="Forecast daily retail sales with strong weekly seasonality"
)

print("\n" + "="*60)
print("RAG-enhanced workflow generated!")
print("="*60)

**What happened?**

The agent:
1. Analyzed data characteristics (daily frequency, strong seasonality)
2. Retrieved similar examples from knowledge base (e.g., "Retail Daily Sales")
3. Saw that `prophet_reg` worked well in similar cases
4. Boosted confidence for `prophet_reg` (up to +10%)
5. Showed key lessons from similar scenarios

In [ ]:
# Direct RAG API usage
from py_agent.knowledge import ExampleLibrary, RAGRetriever, DEFAULT_LIBRARY_PATH

# Load example library
library = ExampleLibrary(DEFAULT_LIBRARY_PATH)
print(f"\n📚 Knowledge Base: {len(library)} examples loaded")

# Create retriever
retriever = RAGRetriever(library)

# Retrieve similar examples
results_rag = retriever.retrieve(
    query="Daily sales data with strong weekly seasonality and promotional effects",
    top_k=3
)

print("\n🔍 Similar Examples:")
for i, result in enumerate(results_rag, 1):
    print(f"\n{i}. {result.example.title} (similarity: {result.similarity_score:.2f})")
    print(f"   Domain: {result.example.domain}")
    print(f"   Recommended: {result.example.recommended_models[:3]}")
    print(f"   Key Lesson: {result.example.key_lessons[0]}")

## Phase 3.5: Autonomous Iteration

Let the agent **autonomously improve** workflows until target performance is reached.

The agent will:
1. Try initial workflow
2. Evaluate performance
3. Diagnose issues (overfitting, underfitting, etc.)
4. Try different approach (regularization, simpler model, tree-based, etc.)
5. Repeat until target reached or max iterations

In [ ]:
# Autonomous iteration until RMSE < 150
best_workflow_iter, history = agent.iterate(
    data=train,
    request="Forecast daily sales with seasonality",
    target_metric='rmse',
    target_value=150.0,  # Stop when RMSE < 150
    max_iterations=5,
    test_data=test
)

print("\n" + "="*60)
print("Autonomous iteration complete!")
print("="*60)

In [ ]:
# Analyze iteration history
print("\n📊 Iteration History:")
print("="*60)

for i, result in enumerate(history, 1):
    status = "✓" if result.success else "✗"
    rmse = result.performance.get('rmse', float('inf'))
    
    print(f"\n{i}. {status} {result.approach}")
    if result.success:
        print(f"   RMSE: {rmse:.2f}")
        print(f"   MAE:  {result.performance.get('mae', 0):.2f}")
        print(f"   R²:   {result.performance.get('r_squared', 0):.4f}")
        if result.issues:
            print(f"   Issues: {', '.join([issue['type'] for issue in result.issues])}")
    else:
        print(f"   Error: {result.error}")
    print(f"   Duration: {result.duration:.1f}s")

# Best performance
best_rmse = min(r.performance.get('rmse', float('inf')) for r in history if r.success)
print(f"\n🏆 Best RMSE achieved: {best_rmse:.2f}")
print(f"Total iterations: {len(history)}")

In [ ]:
# Visualize iteration improvements
iterations = [r.iteration_num for r in history if r.success]
rmses = [r.performance.get('rmse', 0) for r in history if r.success]

plt.figure(figsize=(10, 6))
plt.plot(iterations, rmses, marker='o', linewidth=2, markersize=8)
plt.axhline(y=150, color='red', linestyle='--', label='Target RMSE = 150')
plt.xlabel('Iteration')
plt.ylabel('RMSE')
plt.title('Phase 3.5: Autonomous Iteration - Performance Improvement')
plt.grid(True, alpha=0.3)
plt.legend()
plt.show()

In [ ]:
# Final predictions from best iteration
if best_workflow_iter is not None:
    outputs_iter, _, stats_iter = best_workflow_iter.extract_outputs()
    test_outputs_iter = outputs_iter[outputs_iter['split'] == 'test']
    
    plt.figure(figsize=(14, 6))
    plt.plot(test['date'].values, test_outputs_iter['actuals'].values, label='Actual', alpha=0.7)
    plt.plot(test['date'].values, test_outputs_iter['fitted'].values, label='Best Iteration Prediction', alpha=0.7)
    plt.xlabel('Date')
    plt.ylabel('Sales')
    plt.title('Phase 3.5: Best Autonomous Iteration - Predictions vs Actuals')
    plt.legend()
    plt.grid(True, alpha=0.3)
    plt.show()

## Summary: All Features in One Workflow

Let's combine everything: RAG + Multi-Model + Autonomous Iteration

In [ ]:
# Initialize agent with all features
agent_full = ForecastAgent(verbose=True, use_rag=True)

# Option 1: Quick single workflow (with RAG)
workflow_quick = agent_full.generate_workflow(
    data=train,
    request="Forecast daily sales"
)
print("\n✅ Option 1: Quick workflow generated with RAG")

In [ ]:
# Option 2: Compare multiple models (with RAG)
results_full = agent_full.compare_models(
    data=train,
    request="Forecast daily sales",
    n_models=3,
    cv_strategy='time_series',
    n_folds=3,
    date_column='date'
)
print(f"\n✅ Option 2: Best model from comparison: {results_full['best_model_id']}")

In [ ]:
# Option 3: Autonomous iteration (with RAG)
best_full, history_full = agent_full.iterate(
    data=train,
    request="Forecast daily sales",
    target_metric='rmse',
    target_value=140.0,
    max_iterations=3,
    test_data=test
)
print(f"\n✅ Option 3: Autonomous iteration achieved best RMSE after {len(history_full)} iterations")

## Key Takeaways

### Phase 1: Basic Workflow Generation
- ✅ **Cost**: $0 (rule-based, no API calls)
- ✅ **Speed**: <1 second
- ✅ **Success Rate**: 70-80%
- ✅ **Use Case**: Quick prototyping, simple forecasting tasks

### Phase 3.3: Multi-Model Comparison
- ✅ **Time Savings**: 1-2 hours → 5 minutes (96% reduction)
- ✅ **Models**: Compare 5+ models in parallel
- ✅ **Validation**: Robust CV performance estimates
- ✅ **Use Case**: When you're unsure which model to use

### Phase 3.4: RAG Knowledge Base
- ✅ **Learning**: See similar forecasting examples automatically
- ✅ **Speed**: Sub-100ms retrieval with caching
- ✅ **Confidence**: Models from similar cases get +10% boost
- ✅ **Use Case**: Benefit from past forecasting successes

### Phase 3.5: Autonomous Iteration
- ✅ **Autonomous**: Tries multiple approaches automatically
- ✅ **Self-Debugging**: Detects overfitting, underfitting, etc.
- ✅ **Performance**: Stops when target reached
- ✅ **Use Case**: When you have a specific performance goal

### Overall Achievement
- 🎯 **23 models supported** (baseline → time series → hybrid)
- 🎯 **51 preprocessing steps** with intelligent selection
- 🎯 **90-95% success rate** (from 70% in Phase 1)
- 🎯 **252+ tests passing** (production-ready)
- 🎯 **$0-10 cost** (rule-based free, LLM optional)

## Next Steps

1. **Try your own data**: Replace the sample data with your real forecasting problem
2. **Experiment with constraints**: Add `constraints={'interpretability': 'high'}` for simpler models
3. **Explore all 23 models**: Check `agent.last_workflow_info` to see what models are available
4. **Add domain knowledge**: Use `domain='retail'` in recipe generation for domain-specific preprocessing
5. **Fine-tune iteration**: Adjust `target_value` and `max_iterations` for your performance goals

**Documentation**: See `py_agent/README.md` for complete API reference